In [2]:
%pip install requests
%pip install PyGithub
%pip install python-gitlab
%pip install gitpython
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import requests
from github import Github
from gitlab import Gitlab
from dotenv import load_dotenv
import os

load_dotenv()
# Configuration
GITLAB_URL = os.getenv('GITLAB_URL')
GITLAB_TOKEN = os.getenv('GITLAB_TOKEN')
GITHUB_TOKEN = os.getenv('GITHUB_TOKEN')
GITLAB_PROJECT_ID = os.getenv('GITLAB_PROJECT_ID')
GUTHUB_ORG_NAME = os.getenv('GUTHUB_ORG_NAME')
GITHUB_REPO_NAME = os.getenv('GITHUB_REPO_NAME')

print(f"GITLAB_URL: {GITLAB_URL}")
print(f"GITLAB_TOKEN: {GITLAB_TOKEN}")
print(f"GITHUB_TOKEN: {GITHUB_TOKEN}")
print(f"GITLAB_PROJECT_ID: {GITLAB_PROJECT_ID}")
print(f"GUTHUB_ORG_NAME: {GUTHUB_ORG_NAME}")
print(f"GITHUB_REPO_NAME: {GITHUB_REPO_NAME}")



In [ ]:



# Initialize GitLab and GitHub clients
gitlab_client = Gitlab(GITLAB_URL, private_token=GITLAB_TOKEN)
github_client = Github(GITHUB_TOKEN)
# Get GitLab project
gitlab_project = gitlab_client.projects.get(GITLAB_PROJECT_ID)

# Create GitHub repository
user = github_client.get_user()


In [ ]:
# Check if the repository already exists and delete it if it does
try:
    github_repo = github_client.get_repo(f"{GUTHUB_ORG_NAME}/{GITHUB_REPO_NAME}")
    github_repo.delete()
    print(f"Deleted existing repository: {GITHUB_REPO_NAME}")
except Exception as e:
    print(f"Repository {GITHUB_REPO_NAME} does not exist or could not be deleted: {e}")

# Create a new GitHub repository
org = github_client.get_organization(GUTHUB_ORG_NAME)
github_repo = org.create_repo(GITHUB_REPO_NAME, private=True)
#github_repo = user.create_repo(GITHUB_REPO_NAME, private=True)
repo_url = gitlab_project.http_url_to_repo.replace('https://', f'https://oauth2:{GITLAB_TOKEN}@')


Repository mynetcoreproject does not exist or could not be deleted: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository", "status": "404"}


In [ ]:
import os
from git import Repo


# Migrate repository content
# Clone the GitLab repository
repo_name = GITLAB_PROJECT_ID.split('/')[-1]
if not os.path.exists(repo_name):
    Repo.clone_from(repo_url, repo_name, mirror=True)


# Push the repository to GitHub
github_repo_url = f'https://{GITHUB_TOKEN}@github.com/{GUTHUB_ORG_NAME}/{GITHUB_REPO_NAME}.git'
repo = Repo(repo_name)
repo.remotes.origin.set_url(github_repo_url)
repo.remotes.origin.push(mirror=True)

print("Repository content migrated successfully.")


Repository content migrated successfully.


In [ ]:
# Migrate open pull requests  
import json  
merge_requests = gitlab_project.mergerequests.list()  
for mr in merge_requests:
    print(f"Merge Request ID: {mr.id}, Title: {mr.title}, State: {mr.state}")

    # Check if the merge request is a draft
    if mr.work_in_progress:
        print(f"Merge Request ID: {mr.id} is a draft.")
    else:
        print(f"Merge Request ID: {mr.id} is not a draft.")


In [ ]:

  
with open('usermapping.json', 'r') as file:  
    username_mapping = json.load(file)  

for mr in merge_requests: 
    # Get the author email from the merge request  
    githubuser = username_mapping.get(mr.author['username'])  
    # Load GitHub tokens from the file
    with open('githubtokens.json', 'r') as token_file:
        github_tokens = json.load(token_file)
    
    # Get the token for the mapped GitHub user
    user_token = github_tokens.get(githubuser)
    
    if not user_token:
        print(f"No token found for GitHub user: {githubuser}")
        continue
    
    #print(f"For GitHub user: {githubuser} using {user_token}")
    # Initialize a new GitHub client with the user token
    user_github_client = Github(user_token)
    
    user_github_repo = user_github_client.get_repo(f"{GITHUB_ORG_NAME}/{GITHUB_REPO_NAME}")
        
    # Check if the pull request already exists  
    existing_prs = user_github_repo.get_pulls(state='open', head=f"{user_github_repo.owner.login}:{mr.source_branch}", base=mr.target_branch)  
    if existing_prs.totalCount > 0:  
        user_github_pr = existing_prs[0]  
        user_github_pr.edit(title=mr.title, body=mr.description)  
        print(f"Updated existing pull request: {user_github_pr.html_url}")  
    else:  
        # Create the pull request on GitHub  
        try:  
            user_github_pr = user_github_repo.create_pull(  
                title=mr.title,  
                body=mr.description,  
                head=f"{user_github_repo.owner.login}:{mr.source_branch}",  
                base=mr.target_branch,  
                maintainer_can_modify=True  
            )  
            print(f"Created new pull request: {user_github_pr.html_url}")  
        except Exception as e:  
            print(f"Error creating pull request: {e}")  
            if mr.state in ['closed', 'merged']:
                # Create a dummy commit in the source branch
                dummy_commit_message = "Dummy commit to trigger PR creation"
                dummy_commit = user_github_repo.create_file(
                    path="dummy_file.txt",
                    message=dummy_commit_message,
                    content="This is a dummy commit.",
                    branch=mr.source_branch
                )
                print(f"Created dummy commit: {dummy_commit['commit'].sha}")
                try:
                    user_github_pr = user_github_repo.create_pull(  
                        title=mr.title,  
                        body=mr.description,  
                        head=f"{user_github_repo.owner.login}:{mr.source_branch}",  
                        base=mr.target_branch,  
                        maintainer_can_modify=True  
                    )  
                    print(f"Created new pull request after dummy commit: {user_github_pr.html_url}")  
                except Exception as e:
                    print(f"Error creating pull request after dummy commit: {e}")
            continue  # Skip this merge request if pull request creation fails  
  
    # Migrate comments
    
    discussions = mr.discussions.list()  

    # Sort discussions by date created
    discussions.sort(key=lambda d: d.attributes['notes'][0]['created_at'])
    
    # Order discussions by date created
    for discussion in discussions:  
        
        # Select the first note in the discussion, the one with the lowest id
        first_note = min(discussion.attributes['notes'], key=lambda n: n['id'])
        #print(first_note)
         
        #for emoji in award_emojis:  
            #print(f"{emoji.name} by {emoji.user['username']}")
        first_note_author_name = first_note["author"]['name']  
        first_note_author_username = first_note["author"]['username']  
        first_note_comment_body = first_note['body'] 
        first_note_github_user = username_mapping.get(first_note_author_username)
        
        #print(first_note_comment_body)
        with open('githubtokens.json', 'r') as token_file:
                github_tokens = json.load(token_file)

                # Get the token for the mapped GitHub user
        first_note_user_token = github_tokens.get(first_note_github_user)
        first_note_user_github_client = Github(first_note_user_token)
        first_note_user_github_repo = first_note_user_github_client.get_repo(f"{GITHUB_ORG_NAME}/{GITHUB_REPO_NAME}")
        first_note_user_github_pr = first_note_user_github_repo.get_pull(user_github_pr.number)
        if(first_note['type'] == 'DiscussionNote'):
            print(f'processing note: {first_note_comment_body} as DiscussionNote')
            comment = first_note_user_github_pr.create_issue_comment(first_note_comment_body)    

            # Create a list of notes excluding the first note
            remaining_notes = sorted([note for note in discussion.attributes['notes'] if note['id'] != first_note['id']], key=lambda n: n['id'])
            #print(first_note['type'])
            for note in remaining_notes:  

                # Get the GitHub username for the note author
                author_name = note["author"]['name']  
                author_username = note["author"]['username']  
                comment_body = note['body'] 
                note_github_user = username_mapping.get(author_username)

                # Load GitHub tokens from the file
                with open('githubtokens.json', 'r') as token_file:
                    github_tokens = json.load(token_file)

                # Get the token for the mapped GitHub user
                note_user_token = github_tokens.get(note_github_user)

                if not note_user_token:
                    print(f"No token found for GitHub user: {note_github_user}")
                    continue

                # Initialize a new GitHub client with the user token
                
                note_user_github_client = Github(note_user_token)
                note_user_github_repo = note_user_github_client.get_repo(f"{GITHUB_ORG_NAME}/{GITHUB_REPO_NAME}")
                note_user_github_pr = note_user_github_repo.get_pull(user_github_pr.number)

                #comment = note_user_github_pr.create_issue_comment(f"{author_name} ({author_username}): {comment_body}")    
                # Reply to the first note comment
                reply_body = f"> {first_note_comment_body} \n\n {comment_body}"
                reply_comment = note_user_github_pr.create_issue_comment(body=reply_body)
        elif first_note['type'] == 'DiffNote':
            print(f'processing note: {first_note_comment_body} as Diffnote')
            path = first_note['position']['new_path']
            position = first_note['position']['new_line']
            comment_body = first_note['body']  
            commit_id = first_note['position']['head_sha']
            path = first_note['position']['old_path'] 
            commit = first_note_user_github_repo.get_commit(commit_id)
            comment = first_note_user_github_pr.create_review_comment(
                body=comment_body,
                commit=commit,
                path=path,
                line=first_note['position']['new_line'],
                #side='RIGHT'
            ) 
            remaining_diff_notes = sorted([note for note in discussion.attributes['notes'] if note['id'] != first_note['id']], key=lambda n: n['id'])
            #print(first_note['type'])
            for diff_note in remaining_diff_notes:  
                author_name = diff_note["author"]['name']  
                author_username = diff_note["author"]['username']  
                comment_body = diff_note['body'] 
                note_github_user = username_mapping.get(author_username)

                # Load GitHub tokens from the file
                with open('githubtokens.json', 'r') as token_file:
                    github_tokens = json.load(token_file)

                # Get the token for the mapped GitHub user
                note_user_token = github_tokens.get(note_github_user)

                if not note_user_token:
                    print(f"No token found for GitHub user: {note_github_user}")
                    continue

                note_user_github_client = Github(note_user_token)
                note_user_github_repo = note_user_github_client.get_repo(f"{GITHUB_ORG_NAME}/{GITHUB_REPO_NAME}")
                note_user_github_pr = note_user_github_repo.get_pull(user_github_pr.number)
                note_user_github_pr.create_review_comment_reply(comment.id, diff_note['body'])
        elif first_note['type'] is None:
            print(f'processing note: {first_note_comment_body} as None')
            comment_body = first_note['body']  
            comment = first_note_user_github_pr.create_issue_comment(comment_body)
        
    # Map GitLab merge request state to GitHub pull request state
    state_mapping = {
        'opened': 'open',
        'closed': 'closed',
        'merged': 'closed'
    }

    # Update the state of the pull request on GitHub
    github_pr_state = state_mapping.get(mr.state, 'open')
    print(f"GitLab Merge Request State: {mr.state}, GitHub Pull Request State: {github_pr_state}")
    if user_github_pr.state != github_pr_state:
        user_github_pr.edit(state=github_pr_state)
        print(f"Updated pull request state to: {github_pr_state}")

Updated existing pull request: https://github.com/ivbeljanOrg/mynetcoreproject/pull/1
evo njizi


AttributeError: 'ProjectMergeRequestDiscussion' object has no attribute 'awardemojis'

<class 'gitlab.v4.objects.discussions.ProjectMergeRequestDiscussion'>
was created via a list() call and only a subset of the data may be
present. To ensure all data is present get the object using a
get(object.id) call. For more details, see:

https://python-gitlab.readthedocs.io/en/v4.12.2/faq.html#attribute-error-list

In [8]:
# Delete all comments from the GitHub pull request
comments = user_github_pr.get_issue_comments()
for comment in comments:
    comment.delete()
    print(f"Deleted comment with ID: {comment.id}")

print("All comments deleted successfully.")
# Delete all review comments from the GitHub pull request
review_comments = user_github_pr.get_review_comments()
for review_comment in review_comments:
    review_comment.delete()
    print(f"Deleted review comment with ID: {review_comment.id}")

print("All review comments deleted successfully.")

All comments deleted successfully.
Deleted review comment with ID: 1839967950
Deleted review comment with ID: 1839968047
All review comments deleted successfully.


In [9]:
# Delete the pull request
try:
    user_github_pr.edit(state='closed')
    print(f"Pull request {user_github_pr.number} closed successfully.")
except Exception as e:
    print(f"Error closing pull request: {e}")

Pull request 1 closed successfully.
